<font size="5"><center> <b>Sandpyper: sandy beaches SfM-UAV analysis tools</b></center></font>

    
<center><img src="images/banner.png" width="80%"  /></center>

<font face="Calibri">

# Profile extraction, LoD and data cleaning

<font size="3"> <b> Nicolas Pucino; PhD Student @ Deakin University, Australia </b> <br>

<b>This notebook covers the following concepts:</b>

- The ProfileSet class.
- Data extraction.
- Limit of Detections (LoDs).
</font>


</font>

___

## Introduction

In [4]:
%matplotlib notebook

from sandpyper.profile import ProfileSet
from sandpyper.labels import get_sil_location, get_opt_k

The __ProfileSet__ class is an object that contains all the information necessary to carry a typical *sandpyper* analysis from start to end. Once instantiated, it will contains a number of global variables that store some fundamental monitoring charasteristics such as:

* paths to the data directories
* coordinate reference systems of each site
* location codes and search keywords
* some monitoring parameters such as alongshore transect spacing, acrosshore sampling step and cleaning steps used
* and of course, the elevation and colour profile data.

Moreover, a few key methods will:

1. extract data from the provided transects
2. cluster extracted points using unsupervised clustering algorithm KMeans
3. clean the dataset with user-provided watermasks, shoremasks and fine-tuning polygons.

Moreover, when the __ProfileSet__ class is instantiated, it prints out the number of DSMs and orthophotos found for each location in the provided folders and it also creates a dataframe storing filenames, CRSs, location codes and raw dates extracted from the filenames. This dataframe is useful as a sanity check to see if all the information are correct before proceeding with the actual profile extraction. It will be stored as the attribute *ProfileSet.check*.

Let's see here how it all works...

First, create the required monitoring global settings.

In [10]:
# the paths to the DSM, orthophotos and transect directories
dirNameDSM=r'C:\my_packages\sandpyper\tests\test_data\dsm_1m'
dirNameOrtho=r'C:\my_packages\sandpyper\tests\test_data\orthos_1m'
dirNameTrans=r'C:\my_packages\sandpyper\tests\test_data\transects'

# the location codes used for the monitored locations
loc_codes=["mar","leo"]


# the keyword search dictionary
loc_search_dict = {   'leo': ['St','Leonards','leonards','leo'],
                   'mar': ['Marengo','marengo','mar'] }


# the EPSG codes of the coordinate reference systems for each location code (location) given in CRS string format
crs_dict_string= {
                 'mar': {'init': 'epsg:32754'},
                 'leo':{'init': 'epsg:32755'}
                 }

# the transect spacing of the transects
transects_spacing=20

In [11]:
# create an instance of the ProfileSet class. Let's set 'check' parameter to 'all' to create the check dataframe
# and store it in the variable P.check

P=ProfileSet(dirNameDSM=dirNameDSM,
            dirNameOrtho=dirNameOrtho,
            dirNameTrans=dirNameTrans,
            transects_spacing=transects_spacing,
            loc_codes=loc_codes,
            loc_search_dict=loc_search_dict,
            crs_dict_string=crs_dict_string,
            check="all")

C:\conda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


dsm from leo = 6

ortho from leo = 6

dsm from mar = 9

ortho from mar = 9


NUMBER OF DATASETS TO PROCESS: 30


In [12]:
# Check the infromation extracted from the formatted filenames.
# All the CRSs in a line (survey) must match!

P.check

,raw_date,location,filename_trs_ortho,crs_transect_dsm,filename_raster_dsm,filename_raster_ortho,crs_raster_dsm,crs_raster_ortho
0,20180606,leo,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32755,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32755,32755
1,20180713,leo,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32755,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32755,32755
2,20180920,leo,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32755,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32755,32755
3,20190211,leo,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32755,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32755,32755
4,20190328,leo,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32755,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32755,32755
5,20190731,leo,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32755,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32755,32755
6,20180601,mar,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32754,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32754,32754
7,20180621,mar,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32754,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32754,32754
8,20180727,mar,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32754,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32754,32754
9,20180925,mar,C:\my_packages\sandpyper\tests\test_data\trans...,epsg:32754,C:\my_packages\sandpyper\tests\test_data\dsm_1...,C:\my_packages\sandpyper\tests\test_data\ortho...,32754,32754


## Extraction of profiles from folder

This is the cell where the automatic extraction gets excecuted.
The only parameter to set is the __sampling step__ variable, which indicates the __cross-shore sampling distance (m)__ that we want to use along our transects. Beware, although you could use a very small sampling distance (UAV datasets tend to be between few to 10 cm pixel size), file dimension will increase significantly!.

In [10]:
lod_mode=r"C:\my_packages\sandpyper\tests\test_data\lod_transects"

In [11]:
P.extract_profiles(mode='all',tr_ids='tr_id',sampling_step=1,add_xy=True,lod_mode=lod_mode)

Extracting elevation from DSMs . . .


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Extraction succesfull
Number of points extracted:1395
Time for processing=5.096865653991699 seconds
First 10 rows are printed below
Number of points outside the raster extents: 27
The extraction assigns NaN.
Number of points in NoData areas within the raster extents: 0
The extraction assigns NaN.


__Dealing with NaNs__

>NaNs might come from two different cases:
>1. extraction of points generated on transects falling __outside__ of the underlying raster extent
>2. points sampled from transect __inside__ the raster extent but containing NoData cells.
>
>Conveniently, the extraction profile function makes sure that if points fall outside the raster extent (case 1), those >elevations are assigned a default nan value, in the NumPy np.nan form.
>In case 2, however, the values extracted depends on the definition of NaNs of the source raster format.

## Unsupervised clustering and labelling

The __get_sil_location__ function will iteratively perform KMeans clustering and Silhouette Analysis with increasing number of clusters (k, specified in the `ks` parameter) for every survey, using the feature set specified in the parameter `feature_set`.

This will return a dataframe with Average Silhouette scores with different k for all surveys, which we use to find sub-optimal number of clusters with __get_opt_k__ function.

Then, with the sub-optimal k, we finally run KMeans with __kmeans_sa__ function on all the surveys to obtain clustered points to visually discriminate between sand and non-sand in a Qgis environment.

In [ ]:
# Run interatively KMeans + SA

feature_set=["band1","band2","band3","distance"]
sil_df=get_sil_location(P.profiles,
                        ks=(2,15), 
                        feature_set=feature_set,
                       random_state=10)

Find sub-optimal k by searching inflexion points where an additional cluster do not considerably degrade the overall clustering performance.

In [ ]:
opt_k=get_opt_k(sil_df, sigma=0 )
opt_k

If we are not satisfied with the sub-optimal k returned by the algorithm, we can manually specify each survey k
by defining a dictionary.

In [ ]:
# Based on our observations on a dataset comprising 87 surveys, 10 clusters (k=10) is generally a good tradeoff.

opt_k={'leo_2018-06-06': 10,
 'leo_2018-07-13': 10,
 'leo_2018-09-20': 10,
 'leo_2019-02-11': 10,
 'leo_2019-03-28': 10,
 'leo_2019-07-31': 10,
 'mar_2018-06-01': 10,
 'mar_2018-06-21': 10,
 'mar_2018-07-27': 10,
 'mar_2018-09-25': 10,
 'mar_2018-11-13': 10,
 'mar_2018-12-11': 10,
 'mar_2019-02-05': 10,
 'mar_2019-03-13': 10,
 'mar_2019-05-16': 10}

or, update one value only. For instance, in mar_2019-05-16 dataset, it is unlikely that 3 clusters are enough.<br>
So, we replace only that value with 10.

Optimised K-Means clustering

With the sub-optimal k dictionary and keeping the same feature set, we finally cluster the dataset.

In [ ]:
feature_set=["band1","band2","band3","distance"]

P.kmeans_sa(opt_k,feature_set)

In [ ]:
opt_k['mar_2019-05-16']=10

## Sand labeling and cleaning

In [12]:
water_dict={'leo_20180606':[0,9,10],
'leo_20180713':[0,3,4,7],
'leo_20180920':[0,2,6,7],
'leo_20190211':[0,2,5],
'leo_20190328':[2,4,5],
'leo_20190731':[0,2,8,6],
'mar_20180601':[1,6],
'mar_20180621':[4,6],
'mar_20180727':[0,5,9,10],
'mar_20180925':[6],
'mar_20181113':[1],
'mar_20181211':[4],
'mar_20190205':[],
'mar_20190313':[],
'mar_20190516':[4,7]}

no_sand_dict={'leo_20180606':[5],
'leo_20180713':[],
'leo_20180920':[],
'leo_20190211':[1],
'leo_20190328':[],
'leo_20190731':[1],
'mar_20180601':[4,5],
'mar_20180621':[3,5],
'mar_20180727':[4,7],
'mar_20180925':[5],
'mar_20181113':[0],
'mar_20181211':[0],
'mar_20190205':[0,5],
'mar_20190313':[4],
'mar_20190516':[2,5]}

veg_dict={'leo_20180606':[1,3,7,8],
'leo_20180713':[1,5,9],
'leo_20180920':[1,4,5],
'leo_20190211':[4],
'leo_20190328':[0,1,6],
'leo_20190731':[3,7],
'mar_20180601':[0,7],
'mar_20180621':[1,7],
'mar_20180727':[1,3],
'mar_20180925':[1,3],
'mar_20181113':[3],
'mar_20181211':[2],
'mar_20190205':[3],
'mar_20190313':[1,5],
'mar_20190516':[0]}

sand_dict={'leo_20180606':[2,4,6],
'leo_20180713':[2,6,8],
'leo_20180920':[3],
'leo_20190211':[3],
'leo_20190328':[3],
'leo_20190731':[4,5],
'mar_20180601':[2,3],
'mar_20180621':[0,2],
'mar_20180727':[2,6,8],
'mar_20180925':[0,4,2],
'mar_20181113':[2,4],
'mar_20181211':[3,1],
'mar_20190205':[1,2,4],
'mar_20190313':[0,2,3],
'mar_20190516':[1,3,6]}

In [ ]:


l_dicts={'no_sand': no_sand_dict,
         'sand': sand_dict,
        'water': water_dict,
        'veg':veg_dict}

In [ ]:
label_corrections_path=r"C:\my_packages\sandpyper\tests\test_data\label_corrections.gpkg"
watermasks_path=r"C:\my_packages\sandpyper\tests\test_data\watermasks.gpkg"
shoremasks_path=r"C:\my_packages\sandpyper\tests\test_data\shoremasks.gpkg"

In [ ]:
P.cleanit(l_dicts=l_dicts,
          watermasks_path=watermasks_path,
          shoremasks_path=shoremasks_path,
          label_corrections_path=label_corrections_path)

In [ ]:
dir_out=r'C:\my_packages\sandpyper\tests\test_data'

name="test"
P.save(name,dir_out)
P

## Conclusion

This lenghty notebook showed what Sandpyper needs to operate and what the input data should look like. In the next notebooks I will show how all this information will be analysed in a few lines of code using Sandpyper.